<a href="https://colab.research.google.com/github/profshai/spark-big-data/blob/main/predict_expenditure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict a customer's total amount expenditure using Regression

In [ ]:
pip install pyspark

     |████████████████████████████████| 212.4MB 69kB/s 
     |████████████████████████████████| 204kB 18.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=8a91d18047a3bf745baaa20c08e332da015f822d6aab23ae6cf423fc99651b6b
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Linear Regression').getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header= True)
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [ ]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [ ]:
for item in data.head(2)[1]:
  print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [ ]:
# The columns we want as features
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App','Time on Website', 'Length of Membership'],
                            outputCol = 'features')
output = assembler.transform(data)

In [ ]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [ ]:
final_data = output.select('features', 'Yearly Amount Spent')

In [ ]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [ ]:
# Split data
train_data, test_data = final_data.randomSplit([0.7, 0.3])
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean| 497.95212280589567|
| stddev|  78.19762993961834|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean|  502.2894331647975|
| stddev|  81.87760656179415|
|    min| 256.67058229005585|
|    max|  712.3963268096637|
+-------+-------------------+



In [ ]:
#Regression model: we don't have to specify the features.
lr = LinearRegression(labelCol='Yearly Amount Spent')
lr_model = lr.fit(train_data)

In [ ]:
test_results = lr_model.evaluate(test_data)

In [ ]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  -4.38437971398065|
|  4.621198178553755|
| 22.514365035969377|
|   2.99268065256922|
| -4.021179121657269|
| -6.986327088151825|
| -9.159308731879435|
| -17.59241480148671|
| -5.335027550954976|
| -2.097534021557692|
|  -4.59921847109473|
|  7.854798170838592|
|    8.2248169155842|
|-2.3964265814704504|
|-2.1525464212676297|
|-10.923768719016948|
| -5.328576173608951|
| -17.06363552871659|
|  4.950375741182256|
|  8.582709413012822|
+-------------------+
only showing top 20 rows



In [ ]:
test_results.rootMeanSquaredError

9.737111821736036

In [ ]:
test_results.r2

0.985766716779917

In [ ]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.8364326747734...|
|[31.0472221394875...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5261978982398...|
|[31.5702008293202...|
|[31.6253601348306...|
|[31.7216523605090...|
|[31.7656188210424...|
|[31.8209982016720...|
|[31.8512531286083...|
|[31.8530748017465...|
|[31.8627411090001...|
|[31.8648325480987...|
|[31.8745516945853...|
|[31.9048571310136...|
|[31.9480174211613...|
|[31.9549038566348...|
+--------------------+
only showing top 20 rows



In [ ]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...|471.88628014097026|
|[31.0472221394875...|387.87620101046764|
|[31.2834474760581...| 569.2667243896981|
|[31.3091926408918...| 429.7280371873644|
|[31.4252268808548...| 534.7878977764192|
|[31.4474464941278...|425.58906918337584|
|[31.5261978982398...|418.25383492421724|
|[31.5702008293202...| 563.5379069428916|
|[31.6253601348306...|381.67192830787917|
|[31.7216523605090...|349.87446065343033|
|[31.7656188210424...|501.15330010670186|
|[31.8209982016720...|416.82048284237476|
|[31.8512531286083...| 464.7674297512142|
|[31.8530748017465...| 461.6815500438224|
|[31.8627411090001...| 558.4506875953143|
|[31.8648325480987...| 450.8150491958306|
|[31.8745516945853...|397.61382041987645|
|[31.9048571310136...| 491.0134929515327|
|[31.9480174211613...|456.97050115171555|
|[31.9549038566348...|431.41517052691415|
+--------------------+------------

End of Notebook!